# Prepare data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/ML3/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/ML3


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision.transforms as transforms
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
import numpy as np
from PIL import Image


# Update the CNNModel to include dropout
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.targets[idx]

        if self.transform:
            img = self.transform(img)

        return img, label


# Define the transformation
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    # transforms.RandomRotation(30),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# Hyper parameters
batch_size = 16
learning_rate = 0.001
num_epochs = 15
split_ratio = 0.8

# Load custom dataset
data = pickle.load(open('Train.pkl', 'rb'))
targets = pd.read_csv('Train_labels.csv', usecols=['class']).to_numpy().flatten()
data = torch.from_numpy(data)
targets = torch.from_numpy(targets)
dataset = CustomDataset(data, targets, transform=data_transforms)

# Split the dataset
train_size = int(split_ratio * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Read test data into a Dataset
test_data = pickle.load(open('Test.pkl', 'rb'))
test_data = torch.from_numpy(test_data)
test_targets = torch.zeros(len(test_data), dtype=torch.long)
test_dataset = CustomDataset(test_data, test_targets, transform=data_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluate the model
correct = 0
total = 0

all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Save the true labels and predictions for confusion matrix calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Calculate the confusion matrix
conf_mat = confusion_matrix(all_labels, all_predictions)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_mat)

# Print the accuracy
print(f'Accuracy of the model on the validation images: {100 * correct / total:.2f}%')




Epoch [1/15], Step [100/3000], Loss: 2.1510
Epoch [1/15], Step [200/3000], Loss: 1.7114
Epoch [1/15], Step [300/3000], Loss: 1.8555
Epoch [1/15], Step [400/3000], Loss: 1.0362
Epoch [1/15], Step [500/3000], Loss: 0.9432
Epoch [1/15], Step [600/3000], Loss: 1.1751
Epoch [1/15], Step [700/3000], Loss: 1.5285
Epoch [1/15], Step [800/3000], Loss: 1.3614
Epoch [1/15], Step [900/3000], Loss: 1.1652
Epoch [1/15], Step [1000/3000], Loss: 0.5864
Epoch [1/15], Step [1100/3000], Loss: 1.1708
Epoch [1/15], Step [1200/3000], Loss: 1.1750
Epoch [1/15], Step [1300/3000], Loss: 0.7567
Epoch [1/15], Step [1400/3000], Loss: 1.2690
Epoch [1/15], Step [1500/3000], Loss: 1.3158
Epoch [1/15], Step [1600/3000], Loss: 0.5160
Epoch [1/15], Step [1700/3000], Loss: 0.4046
Epoch [1/15], Step [1800/3000], Loss: 0.8143
Epoch [1/15], Step [1900/3000], Loss: 0.4616
Epoch [1/15], Step [2000/3000], Loss: 0.6841
Epoch [1/15], Step [2100/3000], Loss: 0.5993
Epoch [1/15], Step [2200/3000], Loss: 0.5642
Epoch [1/15], Step 

In [ ]:
# Make predictions on the test dataset
test_pred = []
with torch.no_grad():
    for data in test_loader:
        inputs = data[0]
        inputs=inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_pred.extend(predicted.cpu().numpy())

# Save the predicted labels to a CSV file
test_ids = [i for i in range(len(test_pred))]
test_df = pd.DataFrame({'id': test_ids, 'class': test_pred})
test_df.to_csv('test_pred.csv', index=False)

In [ ]:
import csv
from google.colab import files
files.download('test_preds.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>